# Sistema de recomendación

Librerías necesarias

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy.sparse import csr_matrix

Creo un Dataframe con las columnas que utilizaré para el sistema

In [2]:
df_movies = pd.read_csv("Movies_ETL.csv")
df_movies_sistema = df_movies[["original_language", "vote_average", "release_year"]]
df_movies_sistema

,original_language,vote_average,release_year
0,en,7.7,1995
1,en,6.9,1995
2,en,6.5,1995
3,en,6.1,1995
4,en,5.7,1995
...,...,...,...
45371,en,5.7,1991
45372,tl,9.0,2011
45373,en,3.8,2003
45374,en,0.0,1917


Realizo un One-Hot-Encoding para la columna original_language

In [3]:
encoder = OneHotEncoder(sparse_output=False)
encoded_lang = encoder.fit_transform(df_movies_sistema[['original_language']])

Normalizo vote_average y release_year

In [4]:
scaler = StandardScaler()
caracteristicas_escaladas = scaler.fit_transform(df_movies_sistema[["vote_average", "release_year"]])

Combino todas las características

In [5]:
caracteristicas = np.hstack([encoded_lang, caracteristicas_escaladas])
caracteristicas

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.08380937,  0.12965267],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.66614235,  0.12965267],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.45730884,  0.12965267],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.95231735,  0.46222255],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -2.93623569, -3.11290368],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -2.93623569,  1.04421985]])

Utilizo el cosine_similarity como modelo

In [6]:
coseno_sim = cosine_similarity(caracteristicas[:10000, ::])

In [7]:
# Creo un mapeo entre títulos y sus índices
title_to_index = pd.Series(df_movies.index, index=df_movies['title']).to_dict()

def recomendar_peliculas(titulo, top_n=5):
    if titulo not in title_to_index:
        return "Título no encontrado en el dataset."
    
    idx = title_to_index[titulo]
    sim_scores = list(enumerate(coseno_sim[idx]))
    
    # Se ordenan las películas basadas en la similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Se obtienen las puntuaciones de las top_n películas
    sim_scores = sim_scores[1:top_n+1]  # Excluyendo el propio título
    movie_indices = [i[0] for i in sim_scores]
    
    # Se obtienen los títulos de las películas recomendadas
    recomendadas = df_movies['title'].iloc[movie_indices].tolist()
    
    return recomendadas


In [8]:
# Ejemplo de uso
titulo_pelicula = "Toy Story"  # Reemplaza con el título deseado
peliculas_recomendadas = recomendar_peliculas(titulo_pelicula)
print(f"Películas recomendadas para '{titulo_pelicula}':")
print(peliculas_recomendadas)

Películas recomendadas para 'Toy Story':
['Heat', 'Braveheart', 'Before Sunrise', 'Institute Benjamenta, or This Dream People Call Human Life', 'Heavy']
